# Xenium human lung

This notebook is used to tokenize the Xenium human lung dataset and generate dataset statistics for figures in nicheformer. 

## Imports and definitions

In [1]:
import scanpy as sc
import squidpy as sq
import anndata as ad
import pandas as pd
import numpy as np
import math
import numba
from scipy.sparse import issparse
from sklearn.utils import sparsefuncs

import pyarrow.parquet as pq
import pyarrow
from os.path import join
from tqdm import tqdm

In [2]:
modality_dict = {
    'dissociated': 3,
    'spatial': 4,}

specie_dict = {
    'human': 5,
    'Homo sapiens': 5,
    'Mus musculus': 6,
    'mouse': 6,}

technology_dict = {
    "merfish": 7,
    "MERFISH": 7,
    "cosmx": 8,
    "NanoString digital spatial profiling": 8,
    "Xenium": 9,
    "10x 5' v2": 10,
    "10x 3' v3": 11,
    "10x 3' v2": 12,
    "10x 5' v1": 13,
    "10x 3' v1": 14,
    "10x 3' transcription profiling": 15, 
    "10x transcription profiling": 15,
    "10x 5' transcription profiling": 16,
    "CITE-seq": 17, 
    "Smart-seq v4": 18,
}

## Paths

In [3]:
BASE_PATH = '/lustre/groups/ml01/projects/2023_nicheformer/data/data_to_tokenize'
DATA_PATH = '/lustre/groups/ml01/projects/2023_nicheformer_data_anna.schaar/spatial/preprocessed/human'
OUT_PATH = '/lustre/groups/ml01/projects/2023_nicheformer_data_anna.schaar/tokenized/nicheformer_downstream/xenium_lung'
GENE_MAPPER_PATH = '/lustre/groups/ml01/projects/2023_nicheformer_data_anna.schaar/concat'

## Tokenization functions

In [4]:
def sf_normalize(X):
    X = X.copy()
    counts = np.array(X.sum(axis=1))
    # avoid zero devision error
    counts += counts == 0.
    # normalize to 10000. counts
    scaling_factor = 10000. / counts

    if issparse(X):
        sparsefuncs.inplace_row_scale(X, scaling_factor)
    else:
        np.multiply(X, scaling_factor.reshape((-1, 1)), out=X)

    return X

@numba.jit(nopython=True, nogil=True)
def _sub_tokenize_data(x: np.array, max_seq_len: int = -1, aux_tokens: int = 30):
    scores_final = np.empty((x.shape[0], max_seq_len if max_seq_len > 0 else x.shape[1]))
    for i, cell in enumerate(x):
        nonzero_mask = np.nonzero(cell)[0]    
        sorted_indices = nonzero_mask[np.argsort(-cell[nonzero_mask])][:max_seq_len] 
        sorted_indices = sorted_indices + aux_tokens # we reserve some tokens for padding etc (just in case)
        if max_seq_len:
            scores = np.zeros(max_seq_len, dtype=np.int32)
        else:
            scores = np.zeros_like(cell, dtype=np.int32)
        scores[:len(sorted_indices)] = sorted_indices.astype(np.int32)
        
        scores_final[i, :] = scores
        
    return scores_final


def tokenize_data(x: np.array, median_counts_per_gene: np.array, max_seq_len: int = None):
    """Tokenize the input gene vector to a vector of 32-bit integers."""

    x = np.nan_to_num(x) # is NaN values, fill with 0s
    x = sf_normalize(x)
    median_counts_per_gene += median_counts_per_gene == 0
    out = x / median_counts_per_gene.reshape((1, -1))

    scores_final = _sub_tokenize_data(out, 4096, 30)

    return scores_final.astype('i4')

## Loading model with right gene ordering

In [5]:
model = sc.read_h5ad(
    f"{BASE_PATH}/model.h5ad"
)

## Technology mean

In [6]:
xenium_mean = np.load(
    f"{BASE_PATH}/xenium_mean_script.npy")

In [7]:
xenium_mean = np.nan_to_num(xenium_mean)
rounded_values = np.where((xenium_mean % 1) >= 0.5, np.ceil(xenium_mean), np.floor(xenium_mean))
xenium_mean = np.where(xenium_mean == 0, 1, rounded_values)
xenium_mean

array([1., 1., 1., ..., 1., 1., 1.])

## Loading Xenium lung data

In [8]:
healthy = sc.read_h5ad(f"{DATA_PATH}/10xgenomics_xenium_lung_non_diseased_add_on.h5ad")
healthy

AnnData object with n_obs × n_vars = 295883 × 392
    obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'x', 'y', 'assay_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'donor_id', 'condition_id', 'tissue_type', 'library_key', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'nicheformer_split', 'niche', 'region'
    var: 'gene_name', 'feature_types', 'genome', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'nicheformer_version', 'schema_version', 'title'

In [9]:
diseased = sc.read_h5ad(f"{DATA_PATH}/10xgenomics_xenium_lung_cancer_add_on.h5ad")
diseased

AnnData object with n_obs × n_vars = 531165 × 392
    obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'x', 'y', 'assay_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'donor_id', 'condition_id', 'tissue_type', 'library_key', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'nicheformer_split', 'niche', 'region'
    var: 'gene_name', 'feature_types', 'genome', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'nicheformer_version', 'schema_version', 'title'

In [10]:
xenium = ad.AnnData.concatenate(healthy, diseased)

/tmp/ipykernel_2426527/4150459201.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  xenium = ad.AnnData.concatenate(healthy, diseased)


In [11]:
xenium

AnnData object with n_obs × n_vars = 827048 × 392
    obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'x', 'y', 'assay_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'donor_id', 'condition_id', 'tissue_type', 'library_key', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'nicheformer_split', 'niche', 'region', 'batch'
    var: 'gene_name', 'feature_types', 'genome', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'

In [12]:
xenium.obsm['spatial'] = np.array(xenium.obs[['x', 'y']])
sq.gr.spatial_neighbors(xenium, radius =25, coord_type = 'generic', library_key='condition_id')
xenium

AnnData object with n_obs × n_vars = 827048 × 392
    obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'x', 'y', 'assay_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'donor_id', 'condition_id', 'tissue_type', 'library_key', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'nicheformer_split', 'niche', 'region', 'batch'
    var: 'gene_name', 'feature_types', 'genome', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'spatial_neighbors'
    obsm: 'spatial'
    obsp: 'spatial_connectivities', 'spatial_distances'

### Data statistic figures 

In [33]:
import seaborn as sns

In [34]:
# copying the data to ensure original one stays clean
adata_figs = adata.copy()

In [35]:
adata_figs.layers['counts'] = adata_figs.X

In [36]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

/home/icb/anna.schaar/miniconda3/envs/nf-data/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


In [37]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

         Falling back to preprocessing with `sc.pp.pca` and default params.


/home/icb/anna.schaar/miniconda3/envs/nf-data/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(

KeyboardInterrupt



In [ ]:
sc.settings.set_figure_params(dpi=300, facecolor='white')

In [ ]:
sc.pl.umap(adata, color='condition_id')

## Concatenation
Next we concatenate the `model` and the `merfish` object to ensure they are in the same order. This ensures we have the same gene ordering in the object.

In [27]:
adata = ad.concat([model, xenium], join='inner', axis=0)
# dropping the first observation 
xenium = adata[1:].copy()
# for memory efficiency <
del adata

In [28]:
xenium

AnnData object with n_obs × n_vars = 827048 × 20310
    obs: 'soma_joinid', 'is_primary_data', 'dataset_id', 'donor_id', 'assay', 'cell_type', 'development_stage', 'disease', 'tissue', 'tissue_general', 'specie', 'technology', 'dataset', 'x', 'y', 'assay_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'condition_id', 'tissue_type', 'library_key', 'organism', 'sex', 'niche', 'region', 'nicheformer_split', 'author_cell_type', 'batch'

In [15]:
xenium.obs = xenium.obs[
    ['assay', 'organism', 'nicheformer_split', 'batch']
]
xenium.obs['modality'] = 'spatial'
xenium.obs['specie'] = xenium.obs.organism

In [16]:
xenium.obs.replace({'specie': specie_dict}, inplace=True)
xenium.obs.replace({'modality': modality_dict}, inplace=True)
xenium.obs.replace({'assay': technology_dict}, inplace=True)

In [17]:
xenium.obs

,assay,organism,nicheformer_split,batch,modality,specie
aaaaaahk-1-0,9,Homo sapiens,train,0,4,5
aaaacfah-1-0,9,Homo sapiens,train,0,4,5
aaaafpoc-1-0,9,Homo sapiens,train,0,4,5
aaaaimpp-1-0,9,Homo sapiens,train,0,4,5
aaaakahi-1-0,9,Homo sapiens,train,0,4,5
...,...,...,...,...,...,...
oilgiagl-1-1,9,Homo sapiens,train,1,4,5
oilgidkk-1-1,9,Homo sapiens,train,1,4,5
oilgkofb-1-1,9,Homo sapiens,train,1,4,5
oilglapp-1-1,9,Homo sapiens,train,1,4,5


## Tokenize data

We know tokenize the dataset. 

In [18]:
xenium

AnnData object with n_obs × n_vars = 827048 × 20311
    obs: 'assay', 'organism', 'nicheformer_split', 'batch', 'modality', 'specie'
    obsm: 'spatial'

In [19]:
# dropping the index as the original index can create issues 
xenium.obs.reset_index(drop=True, inplace=True)
# writing the data
#xenium.write(f"{OUT_PATH}/xenium_human_lung_ready_to_tokenize.h5ad")

In [20]:
obs_xenium = xenium.obs
print('n_obs: ', obs_xenium.shape[0])
N_BATCHES = math.ceil(obs_xenium.shape[0] / 10_000)
print('N_BATCHES: ', N_BATCHES)
batch_indices = np.array_split(obs_xenium.index, N_BATCHES)
chunk_len = len(batch_indices[0])
print('chunk_len: ', chunk_len)

n_obs:  827048
N_BATCHES:  83
chunk_len:  9965


In [21]:
xenium_mean.shape

(20310,)

In [22]:
xenium

AnnData object with n_obs × n_vars = 827048 × 20311
    obs: 'assay', 'organism', 'nicheformer_split', 'batch', 'modality', 'specie'
    obsm: 'spatial'

In [23]:
obs_xenium = obs_xenium.reset_index().rename(columns={'index':'idx'})
obs_xenium['idx'] = obs_xenium['idx'].astype('i8')

In [37]:
for batch in tqdm(range(N_BATCHES)):
    obs_tokens = obs_xenium.iloc[batch*chunk_len:chunk_len*(batch+1)].copy()
    tokenized = tokenize_data(xenium.X[batch*chunk_len:chunk_len*(batch+1)], xenium_mean, 4096)

    obs_tokens = obs_tokens[['assay', 'specie', 'modality', 'idx']]
    # concatenate dataframes
    
    obs_tokens['X'] = [tokenized[i, :] for i in range(tokenized.shape[0])]

    # mix spatial and dissociate data
    obs_tokens = obs_tokens.sample(frac=1)
    
    total_table = pyarrow.Table.from_pandas(obs_tokens)
    
    pq.write_table(total_table, f'{join(OUT_PATH)}/test/tokens-{batch}.parquet',
                    row_group_size=1024,)

100%|███████████████████████████████████████████| 83/83 [03:47<00:00,  2.74s/it]


In [38]:
# checking for the last object whether everything looks accurate 
obs_tokens.head(2)

,assay,specie,modality,idx,X
817818,9,5,4,817818,"[6520, 15202, 10870, 14160, 2363, 15907, 955, ..."
824343,9,5,4,824343,"[10870, 7640, 140, 258, 4837, 3807, 4920, 6724..."


In [39]:
pd.read_parquet(f'{join(OUT_PATH)}/tokens-{batch}.parquet').head(2)

,assay,specie,modality,idx,X
821192,9,5,4,821192,"[7427, 706, 16781, 11654, 5076, 8237, 5924, 86..."
817894,9,5,4,817894,"[401, 5600, 6684, 3616, 9609, 15080, 8299, 397..."


In [33]:
OUT_PATH

'/lustre/groups/ml01/projects/2023_nicheformer_data_anna.schaar/tokenized/nicheformer_downstream/xenium_lung'